In [1]:
#Retrieval augmented generation

import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
os.environ["OPENAI_APIKEY"]= os.getenv("OPENAI_APIKEY")

In [7]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents= SimpleDirectoryReader("data").load_data()
print(documents)
print(documents[0].text)

KeyboardInterrupt: 